In [1]:
from sklearn.neighbors import NearestNeighbors
from sklearn.base import BaseEstimator

In [11]:
class EstimatorKNN(BaseEstimator):
    def __init__(self, n_neighbors=3):
        self.estimator = NearestNeighbors(n_neighbors=n_neighbors, algorithm='ball_tree')

    def fit(self, x, y=None):
        self.estimator.fit(x)
        return self 

    def predict(self, x):
        return [1.0]*len(x) 

    def score(self, x, y=None):
        distances, indices = self.estimator.kneighbors(x)
        scores = distances[:,-1]
        return sum(scores)/len(scores)

    def get_params(self, deep=True):
        return {'param1': self.param1, 'param2': self.param2}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self,parameter, value)
        return self

In [8]:
print("knn-!!")

knn-!!
